In [56]:
import pandas as pd
import os 
import re

In [57]:
path = "/Users/joel/Desktop/SUTD /T5/SDS/MY OWN DATA/combined products done2.xlsx"


In [58]:

data = pd.read_excel(path)

print(data.head())
print(data.info())


         RecordedDate     Products Categories Feedback Categories  \
0 2024-04-05 09:21:00  Investments    Vickers         Application   
1 2024-04-05 09:23:00  Investments    Vickers     Process Related   
2 2024-04-05 09:24:00       Others     Others              Others   
3 2024-04-05 09:26:00       Others     Others              Others   
4 2024-04-05 09:27:00  Investments    Vickers         Application   

                                                 Q2b  Q6a  Q6b  Q10  Q19  
0         The app is easy to use and straightforward  NaN  NaN  NaN  NaN  
1                                   All info linked.  NaN  NaN  NaN  NaN  
2                                                NIL  NaN  NaN  NaN  NaN  
3                                                 na  NaN  NaN  NaN  NaN  
4  I was completely new to this but application w...  NaN  NaN  NaN  NaN  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 604 entries, 0 to 603
Data columns (total 9 columns):
 #   Column               Non-N

In [50]:
import nltk

# Download necessary NLTK resources
nltk.download('punkt')  # For tokenization
nltk.download('wordnet')  # For lemmatization
nltk.download('stopwords')  # For stopwords
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to /Users/joel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/joel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/joel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/joel/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [51]:
# import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import classification_report
# import nltk
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer



# data.dropna(subset=['Q2b', 'Feedback Categories'], inplace=True)
# data.reset_index(drop=True, inplace=True)
# feedback_texts = data['Q2b']
# labels = data['Feedback Categories']

# stop_words = set(stopwords.words('english'))
# lemmatizer = WordNetLemmatizer()
# def preprocess(text):
#     return ' '.join([lemmatizer.lemmatize(word) for word in nltk.word_tokenize(text.lower()) if word.isalnum() and not word in stop_words])

# processed_texts = feedback_texts.apply(preprocess)

# tfidf_vectorizer = TfidfVectorizer(max_features=1000)
# tfidf_matrix = tfidf_vectorizer.fit_transform(processed_texts)

# indices = data.index
# X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(tfidf_matrix, labels, indices, test_size=0.2, random_state=42)

# model = RandomForestClassifier(random_state=42)
# model.fit(X_train, y_train)
# #
# predictions = model.predict(X_test)

# original_texts_for_test = feedback_texts.iloc[idx_test].tolist()

# for text, predicted in zip(original_texts_for_test, predictions):
#     print(f"Text: {text}\nPredicted Label: {predicted}\n")


Text: Prompt in replying my query and efficient when assistance required. 
Predicted Label: Staff Related

Text: Darren started the Financial Planning exercise by going through the products I bought previously and makes sure that I understood the products. He then proceeded to propose products which will make better use of my finances and explains throughly the rationale for his proposal. He also answers all my queries professionally and without hesitation. This shows that he is knowledgeable.
Predicted Label: Staff Related

Text: Very good returns
Predicted Label: Staff Related

Text: 1. Recommend the type of products base on my needs 
Predicted Label: Staff Related

Text: Patient and attention to my request. 
Predicted Label: Staff Related

Text: Alicia contacted me last year to advise me to change the policy as the previous one required me to pay high interest rates monthly. 
Predicted Label: Staff Related

Text: When I saw promo online , I will always double check with Ryan through

In [59]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import pandas as pd

clean_data = data.dropna(subset=['Q2b', 'Products'])  

X = clean_data['Q2b']  
y = clean_data['Products']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('classifier', RandomForestClassifier(random_state=42))
])


param_grid = {
    'tfidf__max_df': [0.5, 0.75, 1.0],     ###basically determines the percentage which u drop the common words eg if 'so' appears 80% and 0.75 is chosen, it will drop 
    'tfidf__ngram_range': [(1, 1), (1, 2)], ###single words or consecutive words
    'classifier__n_estimators': [100, 200,300, 400],  ###number of decision trees
    'classifier__max_features': ['auto', 'sqrt', 'log2'],  ###sees how many features to consider for splitting
    'classifier__max_depth': [None, 10, 20, 30, 40, 50],   ##depth of decision tree 
    'classifier__min_samples_split': [2, 5, 10],           
    'classifier__min_samples_leaf': [1, 2, 4]
}


grid_search = GridSearchCV(pipeline, param_grid, cv=5, verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_score = grid_search.best_score_
y_pred = grid_search.predict(X_test)

results = pd.DataFrame({
    'Text': X_test,
    'Actual Product': y_test,
    'Predicted Product': y_pred
})

print("Best Parameters:", best_params)
print("Best Score:", best_score)
print(results.head())


Fitting 5 folds for each of 3888 candidates, totalling 19440 fits
Best Parameters: {'classifier__max_depth': 50, 'classifier__max_features': 'auto', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 10, 'classifier__n_estimators': 300, 'tfidf__max_df': 0.5, 'tfidf__ngram_range': (1, 1)}
Best Score: 0.6513370555392302
                                                  Text Actual Product  \
374  Prompt in replying my query and efficient when...  DBS Treasures   
588  Darren started the Financial Planning exercise...  DBS Treasures   
522                                  Very good returns  DBS Treasures   
192  1. Recommend the type of products base on my n...   DBS Treasure   
590              Patient and attention to my request.   DBS Treasures   

    Predicted Product  
374     DBS Treasures  
588     DBS Treasures  
522     DBS Treasures  
192     DBS Treasures  
590     DBS Treasures  


In [60]:
print(results.head)


<bound method NDFrame.head of                                                   Text Actual Product  \
374  Prompt in replying my query and efficient when...  DBS Treasures   
588  Darren started the Financial Planning exercise...  DBS Treasures   
522                                  Very good returns  DBS Treasures   
192  1. Recommend the type of products base on my n...   DBS Treasure   
590              Patient and attention to my request.   DBS Treasures   
..                                                 ...            ...   
208  Karis was helpful assisting with providing opt...   DBS Treasure   
340  I had gone through the most difficult 5 years ...  DBS Treasures   
596  I believe she have the above mentioned qualities   DBS Treasures   
600  Listen to my needs and recommending the best p...  DBS Treasures   
391                                                 na         Others   

    Predicted Product  
374     DBS Treasures  
588     DBS Treasures  
522     DBS Treasures

In [61]:

results.to_csv('/Users/joel/Desktop/SUTD /T5/SDS/MY OWN DATA/RandomForestWGridSearchFeedback_Predictions2.csv', index=False)

In [62]:
from sklearn.metrics import accuracy_score, classification_report
test_accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))
print("Classification Report:\n", classification_report(y_test, y_pred))

Test Accuracy: 64.42%
Classification Report:
                precision    recall  f1-score   support

 DBS Treasure       0.50      0.08      0.14        25
DBS Treasures       0.62      0.95      0.75        58
  Investments       0.86      0.67      0.75         9
       Others       1.00      0.33      0.50        12

     accuracy                           0.64       104
    macro avg       0.74      0.51      0.53       104
 weighted avg       0.65      0.64      0.57       104

